# Time Consumption

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "data-cleaning-stability":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/Research/NYU/ML_Lifecycle_Project


In [4]:
import pandas as pd
from tabulate import tabulate
from source.visualizations.imputers_viz import get_data_for_box_plots_for_diff_imputers_and_datasets_for_mixed_exp
from configs.constants import (ACS_INCOME_DATASET, ACS_EMPLOYMENT_DATASET, LAW_SCHOOL_DATASET, GERMAN_CREDIT_DATASET,
                               CARDIOVASCULAR_DISEASE_DATASET, BANK_MARKETING_DATASET, DIABETES_DATASET)

## Define global configs

In [5]:
from source.custom_classes.database_client import DatabaseClient, get_secrets_path

db_client_1 = DatabaseClient()
db_client_3 = DatabaseClient(secrets_path=get_secrets_path('secrets_3.env'))
db_client_1.connect()
db_client_3.connect()

In [6]:
DATASETS_ALL_SENSITIVE_ATTRS = {
    ACS_INCOME_DATASET: ['SEX', 'RAC1P', 'SEX&RAC1P'],
    LAW_SCHOOL_DATASET: ['male', 'race', 'male&race'],
    GERMAN_CREDIT_DATASET: ['sex', 'age', 'sex&age'],
    CARDIOVASCULAR_DISEASE_DATASET: ['gender'],
    BANK_MARKETING_DATASET: ['age'],
    DIABETES_DATASET: ['Gender'],
    ACS_EMPLOYMENT_DATASET: ['SEX', 'RAC1P', 'SEX&RAC1P'],
}

DATASETS_SENSITIVE_ATTRS = {
    ACS_INCOME_DATASET: 'SEX&RAC1P',
    LAW_SCHOOL_DATASET: 'male&race',
    GERMAN_CREDIT_DATASET: 'sex',
    CARDIOVASCULAR_DISEASE_DATASET: 'gender',
    BANK_MARKETING_DATASET: 'age',
    DIABETES_DATASET: 'Gender',
    ACS_EMPLOYMENT_DATASET: 'SEX&RAC1P',
}

DATASET_TO_COLUMN_NAME = {
    DIABETES_DATASET: {'cat': ['Family_Diabetes', 'PhysicallyActive', 'RegularMedicine'], 'num': ['SoundSleep']},
    GERMAN_CREDIT_DATASET: {'cat': ['checking-account', 'savings-account', 'employment-since'], 'num': ['duration', 'credit-amount']},
    ACS_INCOME_DATASET: {'cat': ['SCHL', 'MAR'], 'num': ['AGEP', 'WKHP']},
    LAW_SCHOOL_DATASET: {'cat': ['fam_inc', 'tier'], 'num': ['zfygpa', 'ugpa']},
    BANK_MARKETING_DATASET: {'cat': ['education', 'job'], 'num': ['balance', 'campaign']},
    CARDIOVASCULAR_DISEASE_DATASET: {'cat': ['cholesterol', 'gluc'], 'num': ['weight', 'height']},
    ACS_EMPLOYMENT_DATASET: {'cat': ['SCHL', 'DIS', 'MIL'], 'num': ['AGEP']},
}

## Metric Visualizations

In [7]:
def get_imputer_time_data(missingness_types: list, dataset_to_column_name: dict, imputation_quality_metric_name: str,
                          db_client_1, db_client_3, dataset_to_group: dict = None, without_dummy: bool = False):
    imputation_quality_metrics_df = pd.DataFrame()
    for missingness_type in missingness_types:
        train_injection_scenario, test_injection_scenario = missingness_type['train'], missingness_type['test']
        if train_injection_scenario != 'mixed_exp':
            train_injection_scenario = train_injection_scenario.upper()
        test_injection_scenario = test_injection_scenario.upper()

        db_client = db_client_3 if train_injection_scenario == 'mixed_exp' else db_client_1
        imputation_quality_metrics_sub_df, _ = (
            get_data_for_box_plots_for_diff_imputers_and_datasets_for_mixed_exp(train_injection_scenario=train_injection_scenario,
                                                                                test_injection_scenario=test_injection_scenario,
                                                                                metric_name=imputation_quality_metric_name,
                                                                                dataset_to_column_name=dataset_to_column_name,
                                                                                db_client=db_client,
                                                                                dataset_to_group=dataset_to_group,
                                                                                without_dummy=without_dummy))

        imputation_quality_metrics_sub_df['Missingness_Type'] = train_injection_scenario + ' - ' + test_injection_scenario
        imputation_quality_metrics_df = pd.concat([imputation_quality_metrics_df, imputation_quality_metrics_sub_df])

        print(f'Extraction for {missingness_type} is completed\n\n')

    return imputation_quality_metrics_df

In [8]:
imputation_quality_metrics_df = get_imputer_time_data(missingness_types=[
                                                         {'train': 'MCAR3', 'test': 'MCAR3'},
                                                         {'train': 'MAR3', 'test': 'MAR3'},
                                                         {'train': 'MNAR3', 'test': 'MNAR3'},
                                                         {'train': 'mixed_exp', 'test': 'MCAR1 & MAR1 & MNAR1'}
                                                      ],
                                                      dataset_to_column_name=DATASET_TO_COLUMN_NAME,
                                                      imputation_quality_metric_name='runtime_in_mins',
                                                      db_client_1=db_client_1,
                                                      db_client_3=db_client_3)

Extracted data for german
Extracted data for bank
Extracted data for heart
Extracted data for diabetes
Extracted data for law_school
Extracted data for folk
Extracted data for folk_emp
Extraction for {'train': 'MCAR3', 'test': 'MCAR3'} is completed


Extracted data for german
Extracted data for bank
Extracted data for heart
Extracted data for diabetes
Extracted data for law_school
Extracted data for folk
Extracted data for folk_emp
Extraction for {'train': 'MAR3', 'test': 'MAR3'} is completed


Extracted data for german
Extracted data for bank
Extracted data for heart
Extracted data for diabetes
Extracted data for law_school
Extracted data for folk
Extracted data for folk_emp
Extraction for {'train': 'MNAR3', 'test': 'MNAR3'} is completed


Extracted data for german
Extracted data for bank
Extracted data for heart
Extracted data for diabetes
Extracted data for law_school
Extracted data for folk
Extracted data for folk_emp
Extraction for {'train': 'mixed_exp', 'test': 'MCAR1 & MAR1 & MN

In [9]:
imputation_quality_metrics_df.head()

,Dataset_Name,Null_Imputer_Name,Evaluation_Scenario,Experiment_Seed,Dataset_Part,Sample_Size,Kl_Divergence_Pred,Kl_Divergence_Total,Rmse,Precision,Recall,F1_Score,Runtime_In_Mins,Null_Imputer_Params_Dct,Missingness_Type
0,bank,automl,exp1_mcar3,100,X_test_MCAR3,621.0,1.941451,0.001093,NaN,0.554362,0.554362,0.554362,259.697079,NaN,MCAR3 - MCAR3
1,bank,automl,exp1_mcar3,200,X_test_MCAR3,571.0,2.919792,0.001259,NaN,0.559408,0.559408,0.559408,240.951897,NaN,MCAR3 - MCAR3
2,bank,automl,exp1_mcar3,300,X_test_MCAR3,574.5,1.918285,0.000813,NaN,0.510851,0.510851,0.510851,219.435821,NaN,MCAR3 - MCAR3
3,bank,automl,exp1_mcar3,400,X_test_MCAR3,592.5,1.680980,0.000946,NaN,0.547457,0.547457,0.547457,231.788785,NaN,MCAR3 - MCAR3
4,bank,automl,exp1_mcar3,500,X_test_MCAR3,578.5,0.295556,0.000444,NaN,0.557334,0.557334,0.557334,254.405097,NaN,MCAR3 - MCAR3


In [10]:
imputation_quality_metrics_df.shape

(2015, 15)

In [11]:
imputation_quality_metrics_df['Runtime_In_Secs'] = imputation_quality_metrics_df['Runtime_In_Mins'] * 60.0
imp_runtime_df = (
    imputation_quality_metrics_df[
        ['Dataset_Name', 'Null_Imputer_Name', 'Evaluation_Scenario',
         'Experiment_Seed', 'Runtime_In_Secs', 'Missingness_Type']
    ].groupby(['Dataset_Name', 'Null_Imputer_Name', 'Missingness_Type'])
    .agg(mean_runtime=('Runtime_In_Secs', 'mean'), std_runtime=('Runtime_In_Secs', 'std'))
    .groupby(['Dataset_Name', 'Null_Imputer_Name'])
    .agg(avg_mean_runtime=('mean_runtime', 'mean'), avg_std_runtime=('std_runtime', 'mean'))
    .reset_index()
)

In [12]:
imp_runtime_df.shape

(84, 4)

In [13]:
imp_runtime_df.head(20)

,Dataset_Name,Null_Imputer_Name,avg_mean_runtime,avg_std_runtime
0,bank,automl,13893.074535,1687.444203
1,bank,datawig,5089.524323,651.516873
2,bank,deletion,0.045568,0.000603
3,bank,gain,1964.192909,36.911565
4,bank,k_means_clustering,998.505111,49.659836
5,bank,median-dummy,0.027105,0.000418
6,bank,median-mode,0.031075,0.000670
7,bank,miss_forest,4307.551992,1310.523492
8,bank,mnar_pvae,55.554037,30.966291
9,bank,nomi,29.776917,1.489259


In [18]:
# Pivot the DataFrame to organize datasets on X-axis and imputers on Y-axis
pivot_table = imp_runtime_df.pivot(
    index='Null_Imputer_Name',
    columns='Dataset_Name',
    values=['avg_mean_runtime', 'avg_std_runtime']
)

# Sort X-axis (Datasets) and Y-axis (Imputers)
x_axis_order = ['diabetes', 'german', 'folk', 'law_school', 'bank', 'heart', 'folk_emp']
y_axis_order = ['deletion', 'median-mode', 'median-dummy', 'miss_forest',
                'k_means_clustering', 'datawig', 'automl', 'nomi', 'tdm', 'gain', 'notmiwae', 'mnar_pvae']

# Pivot the table and reindex
pivot_table = pivot_table.reindex(index=y_axis_order,
                                  columns=pd.MultiIndex.from_product([['avg_mean_runtime', 'avg_std_runtime'], x_axis_order]))

# Escape underscores in LaTeX
def escape_latex(text):
    return text.replace("_", "\\_")

# Generate LaTeX table with sorting, escaping, rounding, and bold formatting for each column
latex_data = []
columns = ['\\textbf{Imputer}'] + ['\\textbf{' + escape_latex(col) + '}' for col in x_axis_order]

# Generate rows with bold formatting for minimum values
for imputer in pivot_table.index:
    row = [escape_latex(imputer)]

    for dataset in x_axis_order:
        mean_val = pivot_table.loc[imputer, ('avg_mean_runtime', dataset)]
        std_val = pivot_table.loc[imputer, ('avg_std_runtime', dataset)]
        if pd.notnull(mean_val) and pd.notnull(std_val):
            # Apply custom rounding
            mean_str = f"{int(mean_val):d}" if mean_val > 1 else f"{mean_val:.3f}"
            std_str = f"{int(std_val):d}" if std_val > 1 else f"{std_val:.3f}"
            row.append(f"${mean_str} \\pm {std_str}$")
        else:
            row.append("-")
    latex_data.append(row)

In [19]:
# Use tabulate to generate LaTeX code
latex_table = tabulate(latex_data, headers=columns, tablefmt="latex_raw")

# Output the LaTeX table code
print("Generated LaTeX Table:")
print(latex_table)

Generated LaTeX Table:
\begin{tabular}{llllllll}
\hline
 \textbf{Imputer}     & \textbf{diabetes}   & \textbf{german}   & \textbf{folk}     & \textbf{law\_school}   & \textbf{bank}     & \textbf{heart}    & \textbf{folk\_emp}   \\
\hline
 deletion             & $0.013 \pm 0.000$   & $0.013 \pm 0.001$ & $0.024 \pm 0.001$ & $0.025 \pm 0.000$      & $0.046 \pm 0.001$ & $0.087 \pm 0.004$ & $1 \pm 0.049$        \\
 median-mode          & $0.012 \pm 0.000$   & $0.014 \pm 0.001$ & $0.023 \pm 0.001$ & $0.025 \pm 0.001$      & $0.031 \pm 0.001$ & $0.067 \pm 0.003$ & $0.933 \pm 0.019$    \\
 median-dummy         & $0.013 \pm 0.000$   & $0.014 \pm 0.001$ & $0.021 \pm 0.001$ & $0.024 \pm 0.001$      & $0.027 \pm 0.000$ & $0.053 \pm 0.001$ & $0.773 \pm 0.024$    \\
 miss\_forest         & $111 \pm 17$        & $244 \pm 86$      & $1758 \pm 526$    & $2530 \pm 851$         & $4307 \pm 1310$   & $6337 \pm 1601$   & $20358 \pm 4934$     \\
 k\_means\_clustering & $10 \pm 0.402$      & $13 \pm 0.442$  